In [17]:
import pandas as pd

In [18]:
data=pd.read_csv('/home/siddhant/Desktop/data_sets/fake_news/fake-news/train.csv')

In [19]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [21]:
data.shape

(20800, 5)

In [22]:
data=data.drop(['title','author','id'],axis=1)

In [23]:
data.isnull().sum()

text     39
label     0
dtype: int64

In [24]:
data=data.dropna()

In [25]:
data.isnull().sum()

text     0
label    0
dtype: int64

In [26]:
data.shape

(20761, 2)

In [27]:
data.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [29]:
X=data['text']
y=data['label']

In [31]:
X.shape

(20761,)

In [32]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [33]:
voc_size=5000

In [38]:
X=X.reset_index()

In [42]:
X=X.drop(['index'],axis=1)

In [43]:
X.tail()

,text
20756,Rapper T. I. unloaded on black celebrities who...
20757,When the Green Bay Packers lost to the Washing...
20758,The Macy’s of today grew from the union of sev...
20759,"NATO, Russia To Hold Parallel Exercises In Bal..."
20760,"David Swanson is an author, activist, journa..."


In [45]:
y=y.reset_index()

In [47]:
y=y.drop(['index'],axis=1)

In [48]:
y.tail()

,label
20756,0
20757,0
20758,0
20759,1
20760,1


In [52]:
import nltk
import re
from nltk.corpus import stopwords

In [53]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ',X['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [56]:
corpus[3]

'video civilian kill singl us airstrik identifi rate civilian kill american airstrik afghanistan higher us engag activ combat oper photo hellfir missil load onto us militari reaper drone afghanistan staff sgt brian ferguson u air forc bureau abl identifi civilian kill singl us drone strike afghanistan last month biggest loss civilian life one strike sinc attack medecin san frontier hospit msf last octob us claim conduct counter terror strike islam state fighter hit nangarhar provinc missil septemb next day unit nation issu unusu rapid strong statement say strike kill civilian injur other gather hous celebr tribal elder return pilgrimag mecca bureau spoke man name haji rai said owner hous target said peopl kill other injur provid name list bureau abl independ verifi ident die rai son headmast local school among anoth man abdul hakim lost three son attack rai said involv deni us claim member visit hous strike said even speak sort peopl phone let alon receiv hous death amount biggest conf

In [63]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [64]:
len(onehot_repr[0])

433

In [146]:
len(onehot_repr[700])

85

In [66]:
sent_length=700


embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3312 4692 4109]
 [   0    0    0 ...  591 4400  198]
 [   0    0    0 ... 1962  949 2082]
 ...
 [   0    0    0 ... 2702 3779 4967]
 [   0    0    0 ...  472 3534 1307]
 [   0    0    0 ... 4135 3367 2220]]


In [70]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [71]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [72]:
X_final.shape,y_final.shape

((20761, 700), (20761, 1))

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.10, random_state=42)

In [79]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Train on 18684 samples, validate on 2077 samples
Epoch 1/5
18684/18684 [==============================] - 191s 10ms/sample - loss: 0.2321 - accuracy: 0.9068 - val_loss: 0.2574 - val_accuracy: 0.9066
Epoch 2/5
18684/18684 [==============================] - 201s 11ms/sample - loss: 0.1388 - accuracy: 0.9542 - val_loss: 0.2965 - val_accuracy: 0.8849
Epoch 3/5
18684/18684 [==============================] - 233s 12ms/sample - loss: 0.0908 - accuracy: 0.9684 - val_loss: 0.2264 - val_accuracy: 0.9297
Epoch 4/5
18684/18684 [==============================] - 208s 11ms/sample - loss: 0.0622 - accuracy: 0.9788 - val_loss: 0.2557 - val_accuracy: 0.9244
Epoch 5/5
18684/18684 [==============================] - 181s 10ms/sample - loss: 0.0617 - accuracy: 0.9790 - val_loss: 0.3086 - val_accuracy: 0.8849


In [111]:
test=pd.read_csv('/home/siddhant/Desktop/data_sets/fake_news/fake-news/test.csv')

In [112]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [113]:
test.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [114]:
test=test.drop(['title','id','author'],axis=1)

In [115]:
test.head()

,text
0,"PALO ALTO, Calif. — After years of scorning..."
1,Russian warships ready to strike terrorists ne...
2,Videos #NoDAPL: Native American Leaders Vow to...
3,"If at first you don’t succeed, try a different..."
4,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [116]:
test.isnull().sum()

text    7
dtype: int64

In [117]:
test.fillna('nan value present',inplace=True)

#the solution file that can be submitted in kaggle expects it to have 5200 rows so we can't drop rows in the test dataset

In [119]:
test.shape

(5200, 1)

In [120]:

corpus_test = []
for i in range(0, len(test)):
    review = re.sub('[^a-zA-Z]', ' ',test['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_test.append(review)

In [121]:
onehot_repr_test=[one_hot(words,voc_size)for words in corpus_test] 

In [122]:
sent_length=700


embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=sent_length)
print(embedded_docs_test)

[[  32 4775 1338 ...  509 2005 1240]
 [   0    0    0 ... 3833 2999 2772]
 [   0    0    0 ... 2154 1460 1999]
 ...
 [   0    0    0 ... 2871 3764 3567]
 [   0    0    0 ...  558 4533  919]
 [3608  883  620 ... 2820 1857 2267]]


In [123]:
X_test=np.array(embedded_docs_test)

In [124]:
check=model.predict_classes(X_test)

In [125]:
check

array([[0],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [126]:
check.shape

(5200, 1)

In [127]:
test.shape

(5200, 1)

In [128]:
submit_sample=pd.read_csv('/home/siddhant/Desktop/data_sets/fake_news/fake-news/submit.csv')

In [129]:
submit_sample.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [130]:
type(check)

numpy.ndarray

In [132]:
check[0]

array([0], dtype=int32)

In [133]:
val=[]
for i in check:
    val.append(i[0])

In [138]:
submit_sample['label']=val

In [139]:
submit_sample.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [142]:
submit_sample.to_csv('/home/siddhant/Desktop/data_sets/fake_news/fake-news/sub_third.csv',index=False)